# Metadata Refresh for Snowflake


## Insert Server Name (if needed)

> The 'Metadata Server Name' is a **Friendly** name. It does not need to be the same as the _true_ Server name (which is determined by the 'Connection' selected at runtime)

In [ ]:
DECLARE @server_name NVARCHAR(128) = /*${snowflake_Metadata_Server_Name}*/'snowflake_dev'/**/

IF NOT EXISTS (SELECT 1 FROM [meta].[dim_server] WHERE [server_name] = @server_name)
	BEGIN
		INSERT INTO [meta].[dim_server]
				   ([server_name]
				   ,[server_type]
				   )
			 VALUES
				   ( @server_name
				   , 'snowflake')
	END;

SELECT @@ROWCOUNT AS [row_count];


## Truncate staging tables

> Reusing 'Truncate staging tables' step from 'Metadata Refresh for SQL Server (Project)'

## Retrieve all Databases 
> Using code from refresh_database_names_snowflake.sql file (content\refresh_database_names_snowflake.sql)

## Merge stg to meta schema - dim\_database (snowflake)

In [ ]:
DELETE [meta].[dim_database]
  FROM [meta].[dim_database] AS e
  LEFT JOIN [stg].[dim_database] AS s
    ON s.[server_name] = e.[server_name]
   AND s.[database_name] = e.[database_name]
 WHERE e.[server_name] = '${snowflake_Metadata_Server_Name}'
   AND s.[database_name] IS NULL

INSERT [meta].[dim_database]
SELECT s.[server_name]
	 , s.[database_name]
	 , s.[database_create_date]
	 , s.[change_tracking_enabled]
     , 'N'
     , 'N'
  FROM [stg].[dim_database] AS s
  LEFT JOIN [meta].[dim_database] AS e
    ON e.[server_name] = s.[server_name]
   AND e.[database_name] = s.[database_name]
 WHERE s.[server_name] = '${snowflake_Metadata_Server_Name}'
   AND e.[database_name] IS NULL;

SELECT @@ROWCOUNT AS [row_count];

## src to stg - SQL Server tables

> Using code from src_to_stg_snowflake_tables.sql file (content\src_to_stg_snowflake_tables.sql)

## src to stg - SQL Server columns

> Using code from src_to_stg_snowflake_columns.sql file (content\src_to_stg_snowflake_columns.sql)

## Merge - stg to meta - dim_table

In [ ]:
MERGE [meta].[dim_table] AS DST
USING [stg].[dim_table] AS SRC
   ON src.[server_name] = dst.[server_name]
   AND src.[database_name] = dst.[database_name]
   AND src.[table_schema] = dst.[table_schema]
   AND src.[table_name] = dst.[table_name]

WHEN MATCHED AND (    DST.[table_type] != SRC.[table_type] 
                   OR DST.[has_identity] != SRC.[has_identity] 
	           OR DST.[has_primary_key] != SRC.[has_primary_key] ) THEN
		UPDATE SET [table_type] =  SRC.[table_type] 
                 , [has_identity] =  SRC.[has_identity] 
                 , [has_primary_key] = SRC.[has_primary_key] 

WHEN NOT MATCHED THEN 
INSERT ( [server_name]
      ,[database_name]
      ,[table_schema]
      ,[table_name]
      ,[table_type]
      ,[has_identity]
      ,[has_primary_key]
      ,[enable_change_tracking]
	   )
VALUES (
	   [server_name]
      ,[database_name]
      ,[table_schema]
      ,[table_name]
      ,[table_type]
      ,[has_identity]
      ,[has_primary_key]
      ,'N'	
	   )
WHEN NOT MATCHED BY SOURCE 
 AND DST.[server_name] = '${snowflake_Metadata_Server_Name}' 
 AND DST.[database_name] = /*${sql_server_Database}*/'eltsnap_v2'/**/ THEN
	 DELETE;

SELECT @@ROWCOUNT AS [row_count];

## Merge - stg to meta - dim_column

In [ ]:
MERGE [meta].[dim_column] AS DST
USING [stg].[dim_column] AS SRC
   ON src.[server_name] = dst.[server_name]
   AND src.[database_name] = dst.[database_name]
   AND src.[table_schema] = dst.[table_schema]
   AND src.[table_name] = dst.[table_name]
   AND src.[column_name] = dst.[column_name]

WHEN MATCHED AND (    DST.[ordinal_position] != SRC.[ordinal_position] 
                   OR DST.[is_nullable] != SRC.[is_nullable] 
		   OR DST.[data_type] != SRC.[data_type]
                   OR DST.[char_max_length] != SRC.[char_max_length] 
		   OR DST.[primary_key] != SRC.[primary_key] 
               ) THEN
UPDATE SET [ordinal_position] = SRC.[ordinal_position] 
         , [is_nullable] = SRC.[is_nullable] 
         , [data_type] = SRC.[data_type] 
         , [char_max_length] = SRC.[char_max_length]
		   , [primary_key] = SRC.[primary_key]

WHEN NOT MATCHED THEN 
INSERT (  [server_name]
	    , [database_name]
        , [table_schema]
        , [table_name]
        , [column_name]
        , [ordinal_position]
        , [is_nullable]
        , [data_type]
        , [char_max_length]
		, [primary_key]
	   )
VALUES (
		  [server_name]
	    , [database_name]
        , [table_schema]
        , [table_name]
        , [column_name]
        , [ordinal_position]
        , [is_nullable]
        , [data_type]
        , [char_max_length]
		, [primary_key]
	   )
WHEN NOT MATCHED BY SOURCE
 AND DST.[server_name] = '${snowflake_Metadata_Server_Name}' 
 AND DST.[database_name] = /*${sql_server_Database}*/'eltsnap_v2'/**/ THEN
	 DELETE;

SELECT @@ROWCOUNT AS [row_count];

## Merge stg to dbo schema - dim_table

> Reusing 'Merge stg to dbo schema - dim_table' step from 'Metadata Refresh for SQL Server (Project)'

## Merge stg to dbo schema - dim_column

> Reusing 'Merge stg to dbo schema - dim_column' step from 'Metadata Refresh for SQL Server (Project)'

## dim delete - dbo - dim_table

> Reusing 'dim delete - dbo - dim_table' step from 'Metadata Refresh for SQL Server (Project)'

## dim delete - dbo - dim_column

> Reusing 'dim delete - dbo - dim_column' step from 'Metadata Refresh for SQL Server (Project)'